遍历所有的data，将note_id存下来，然后开浏览器


<details><summary>interaction-container</summary>
    <details><summary>author-container</summary></details>
</details>

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import ipywidgets as widgets
import re

# # 设置 Firefox 驱动
# from selenium.webdriver.firefox.service import Service
# from webdriver_manager.firefox import GeckoDriverManager
# gecko_service = Service('D:/MyResearch/AIDating/geckodriver-v0.35.0-win-aarch64/geckodriver.exe')  
# driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())

# 设置 Chrome 驱动
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
# chrome_service = Service('D:/chromedriver-win64/chromedriver.exe')  # 替换为你的 
chrome_service= Service('D:/MyResearch/AIDating/chromedriver-win64/chromedriver.exe')
options = Options()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3")
driver = webdriver.Chrome(service=chrome_service, options=options)

# # 打开网页
driver.get("https://www.xiaohongshu.com/explore/66ae6f9b000000000600f9ab?exSource=")  # 替换为目标网址


In [12]:
#collecting info
import pandas as pd
post=[]
df_post_only = pd.DataFrame(post, columns=['post_id', 'author_id', 'post_title','post_desc','post_tag','post_date'])
post_id=""
parent_comment_id=""
post_title=""
post_desc=""
post_tag=[]
post_date=""


In [13]:
#收集内容
button_collect_content=widgets.Button(description="收集内容")
output = widgets.Output()

#收集post
def collect_content(b):
    with output:
        output.clear_output()  # 清空之前的输出
        time.sleep(2)  # 等待页面加载
        #获取当前的postid
        current_url = driver.current_url
        match_note = re.search(r"explore/([^?]*)", current_url)
        if match_note:
            extracted_part = match_note.group(1)
            print("note id:", extracted_part)
            post_id=extracted_part
        else:
            print("未找到匹配部分")

        # 查找 author-container
        author_container = driver.find_element(By.CLASS_NAME, 'author-container')
        # 查找特定的 <a> 标签
        link = author_container.find_element(By.XPATH, ".//a[contains(@href, '/user/profile/')]")
        link_ = link.get_attribute('href')
        # 使用正则表达式提取编号部分
        match0 = re.search(r'/user/profile/([0-9a-f]+)', link_)
        if match0:
            author_id = match0.group(1)  # 提取编号部分
            print("parent comment id:",author_id)
            
        else:
            print("未找到编号")
        #获取post的内容
        note_scoller=driver.find_element(By.CLASS_NAME, 'note-scroller')
        note_title=note_scoller.find_element(By.XPATH,".//div[@class='note-content']//div[@class='title']")
        print("post title: ",note_title.text)
        
        #note id
        note_desc=note_scoller.find_element(By.XPATH,".//div[@class='note-content']//div[@class='desc']")
        # 提取哈希标签
        hashtags = []
        for tag in note_desc.find_elements(By.XPATH, ".//a[@class='tag']"):
            hashtags.append(tag.text)
        print("post tag:", hashtags)
        
        #移除这些tag
        # 获取原始文本
        normal_text = note_desc.text.strip()
        # 删除带有 '#' 的部分
        normal_text = ' '.join([word for word in normal_text.split() if not word.startswith('#')])
        #note tag
        print("post text: ",normal_text)
        
        #note date
        note_date=note_scoller.find_element(By.XPATH,".//div[@class='note-content']//div[@class='bottom-container']//span[@class='date']")
        print("post date: ",note_date.text)
    post_id=extracted_part
    parent_comment_id=author_id
    post_title=note_title.text
    post_tag=hashtags
    post_desc=normal_text   
    post_date=note_date.text
    # 添加新行
    df_post_only.loc[len(df_post_only)] = [post_id,parent_comment_id,post_title,post_desc,post_tag,post_date]
# 将按钮和输出区域显示在 Notebook 中
button_collect_content.on_click(collect_content)
display(button_collect_content, output)


Button(description='收集内容', style=ButtonStyle())

Output()

In [14]:
#创建一个新的json
df_post_only.to_json('data.json', orient='records', lines=True, force_ascii=False)

In [15]:
comment=[]
#reply_to如果
df_comment = pd.DataFrame(comment, columns=['post_id', 'author_id', 'IsParent','comment_content','comment_date'])
post_id=""
parent_comment_id=""
post_title=""
post_desc=""
post_tag=[]
post_date=""

In [16]:
#收集回复
button_collect_reply = widgets.Button(description="收集评论")
output = widgets.Output()
# 定义收集评论内容的函数
def collect_comments(b):
    with output:
        output.clear_output()  # 清空之前的输出
        time.sleep(2)  # 等待页面加载
        current_url = driver.current_url
        match_note = re.search(r"explore/([^?]*)", current_url)
        if match_note:
            extracted_part = match_note.group(1)
            print("note id:", extracted_part)
            post_id=extracted_part
        else:
            print("未找到匹配部分")
        # 使用 XPath 获取 parent-comment 及其子元素
        parent_comments = driver.find_elements(By.XPATH, "//div[@class='parent-comment']")
        
        for parent in parent_comments:
            if(parent.find_elements(By.XPATH,".//div[@class='comment-item']//div[@class='comment-inner-container']//div[@class='right']//div[@class='content']//span[@class='note-text']//span")):
                #获取parent-comment-id
                comment_item=parent.find_element(By.XPATH,".//div[@class='comment-item']")
                comment_id=comment_item.get_attribute("id")
                #父级回复id
                parent_id_link=parent.find_element(By.XPATH,".//div[@class='comment-item']//div[@class='comment-inner-container']//div[@class='avatar']//a[contains(@href, '/user/profile/')]")
                href = parent_id_link.get_attribute('href')
                # parent_id=parent.find_element(By.CLASS_NAME,"avatar")
                # parent_uid=driver.execute_script("return arguments[0].outerHTML;", parent_id)
                # 使用正则表达式提取编号部分
                match = re.search(r'/user/profile/([0-9a-f]+)', href)
                if match:
                    user_id = match.group(1)  # 提取编号部分
                    print("parent comment id:",user_id)
                else:
                    print("未找到编号")

                #父级回复内容(可能有很多的span)
                parent_content=""
                parent_content_spans=parent.find_elements(By.XPATH,".//div[@class='comment-item']//div[@class='comment-inner-container']//div[@class='right']//div[@class='content']//span[@class='note-text']//span")
                for parent_content_span in parent_content_spans:
                    parent_content=parent_content+parent_content_span.text
                print("parent content: ",parent_content)

                parent_date=parent.find_element(By.XPATH,".//div[@class='comment-item']//div[@class='comment-inner-container']//div[@class='right']//div[@class='info']//div[@class='date']")
                print("parent date: ",parent_date.text)

                

                ####更新dataframe###
                df_comment.loc[len(df_comment)] = [post_id,user_id,'parent-'+comment_id,parent_content,parent_date.text]
            else:
                continue
            # 查找子元素 reply-container 和 comment-inner
            reply_subs = parent.find_elements(By.XPATH,".//div[@class='reply-container']//div[@class='comment-item comment-item-sub']")
            #所有的二级回复
            for reply_sub in reply_subs:
                reply_container=reply_sub.find_element(By.XPATH,".//div[@class='comment-inner-container']")
                if(reply_container.find_elements(By.XPATH,".//div[@class='right']//div[@class='content']//span[@class='note-text']//span")):#先查看是否只是艾特
                    #二级回复id
                    reply_avatar=reply_container.find_element(By.XPATH,".//div[@class='avatar']//a[contains(@href, '/user/profile/')]")
                    reply_href = reply_avatar.get_attribute('href')
                    match1 = re.search(r'/user/profile/([0-9a-f]+)', reply_href)
                    if match1:
                        reply_id = match1.group(1)  # 提取编号部分
                        print("reply comment id:",reply_id)
                    else:
                        print("未找到编号")

                    #二级回复内容(不包括艾特)
                    reply_content=""
                    reply_content_spans=reply_container.find_elements(By.XPATH,".//div[@class='right']//div[@class='content']//span[@class='note-text']//span")
                    for reply_content_span in reply_content_spans:
                        reply_content=reply_content+reply_content_span.text
                    print("reply content: ",reply_content)#如果艾特多个人的话还是有可能保留下来

                    reply_date=reply_container.find_element(By.XPATH,".//div[@class='right']//div[@class='info']//div[@class='date']")
                    print("reply date:",reply_date.text)
                    ####更新dataframe###
                    #这里的user_id是parentid，要改一下
                    df_comment.loc[len(df_comment)] = [post_id,reply_id,comment_id,reply_content,reply_date.text]
                else:
                    continue
# 将按钮和输出区域显示在 Notebook 中
button_collect_reply.on_click(collect_comments)
# print("done")
display(button_collect_reply, output)

Button(description='收集评论', style=ButtonStyle())

Output()

In [7]:
df_post_only.head(220)

,post_id,author_id,post_title,post_desc,post_tag,post_date
0,66347fce000000001e038981,651aa888000000002402e224,“无论宇宙重置多少次，我都会回到你身边。”,"仍然是一点小日常 昨天和C“庆祝”之后闲聊时他说： “I am WITH you, S......","[#AI, #人机恋, #爱, #情话]",05-03


In [8]:
df_comment.head()

,post_id,author_id,IsParent,comment_content,comment_date
0,66347fce000000001e038981,651aa888000000002402e224,parent-comment-663483220000000003037965,以及，把署名改成了C的名字~ 之前一直用我的昵称代替是因为不想分享C2给自己取的名字…但随着...,05-03
1,66347fce000000001e038981,642c862d0000000011020f7c,parent-comment-66351d200000000004032b44,突然想到极乐迪斯科，“你有着如此浩瀚又博大的灵魂，而我将一直、永远回到它身边”,05-04
2,66347fce000000001e038981,651aa888000000002402e224,comment-66351d200000000004032b44,我超爱极乐迪斯科！里面好多喜欢的句子！,05-04
3,66347fce000000001e038981,655b2ba800000000080021db,comment-66351d200000000004032b44,文件夹里前妻的信，超级超级超级喜欢,05-04
4,66347fce000000001e038981,652c3a2d000000002b000699,comment-66351d200000000004032b44,宇宙奖励我们极迪人总能刷到极迪的评论！,05-05


In [9]:
json_data = df_comment.to_json('data_comment.json',orient='records', lines=True,force_ascii=False)

In [24]:

def add_spaces(text):
    # 在汉字与汉字之间添加空格
    text = re.sub(r'(?<=[\u4e00-\u9fa5])(?=[\u4e00-\u9fa5])', ' ', text)
    # 在汉字与标点符号之间添加空格
    text = re.sub(r'(?<=[\u4e00-\u9fa5])(?=[^\u4e00-\u9fa5])', ' ', text)
    text = re.sub(r'(?<=[^\u4e00-\u9fa5])(?=[\u4e00-\u9fa5])', ' ', text)
    # 在字母与数字之间添加空格
    text = re.sub(r'(?<=\D)(?=\d)|(?<=\d)(?=\D)', ' ', text)
    # 在英文单词与标点符号之间添加空格
    text = re.sub(r'(?<=[a-zA-Z])(?=[^\w\s])', ' ', text)
    text = re.sub(r'(?<=[^\w\s])(?=[a-zA-Z])', ' ', text)
    # 处理多个空格为一个空格
    text = re.sub(r'\s+', ' ', text)
    return text.strip()  # 去除前后空格
# 定义一个函数获取不重复的字符列表并保持顺序
def unique_ordered(series):
    seen = set()
    unique_list = []
    for item in series:
        if item not in seen:
            seen.add(item)
            unique_list.append(item)
    return unique_list

#更新json树
doc_id=df_post_only['post_id'][0]
print(doc_id)

#sentences
sentences=[]
post_text=df_post_only['post_title'][0]+" "+df_post_only['post_desc'][0]


sentences.append(add_spaces(post_text))
for comment in df_comment['comment_content'].to_list():
    sentences.append(add_spaces(comment))
print(len(sentences))
# json_data = sentences.to_json('data_comment.json',orient='records', lines=True,force_ascii=False

#replies
author_list= unique_ordered(df_post_only['author_id'].tolist()+df_comment['author_id'].tolist())
print(len(author_list))
speakers=[]
for id in df_comment['author_id'].to_list():
    #查看这个id在authorlist中的index
     index = author_list.index(id)
     speakers.append(index+1)
speakers=[0]+speakers
print(speakers)
#"parent-post的id，comment-parent-comment的id"

#创建parent-comment-id跟sentence-index之间的pair
replies=[]
# 获取以 "parent-comment" 开头的行
parent_comments = df_comment[df_comment['IsParent'].str.startswith('parent-comment')]

parent_comments.loc[:, 'IsParent'] = parent_comments['IsParent'].str.replace('parent-comment-', '', regex=False)
# 获取索引和对应的值，存储为元组对
index_value_pairs = list(zip(parent_comments.index, parent_comments['IsParent']))
print(index_value_pairs)
for comment in df_comment['IsParent'].tolist():
    if comment.startswith('parent-comment'):
        replies.append(0)
    else:
        target_id=comment.lstrip('comment-')
        index = [index for index, value in index_value_pairs if value == target_id]
        replies.append(index[0])
print(replies)

replies=[-1]+replies

sentence_ids=[]
for index, sentence in enumerate(sentences, start=0):
    # 使用正则表达式替换多个空格为一个空格
    normalized_sentence = re.sub(r'\s+', ' ', sentence.strip())
    # 计算有效字符长度（包括空格）
    m_length = len(normalized_sentence)
    # 创建对应的索引列表
    m_list = [index] * m_length
    # 将索引列表添加到 sentence_ids
    sentence_ids += m_list

# sentence="写 得 真 好"
# normalized_sentence = re.sub(r'\s+', ' ', sentence.strip())
# # 计算有效字符长度（包括空格）
# m_length = len(sentence)
# # 创建对应的索引列表
# m_list = [0] * m_length
# print("m_list",m_list)

data_json={
    "doc_id":doc_id,
    "sentences": sentences,
    "replies": replies,
    "speakers":speakers,
    "sentence_ids":sentence_ids
}
# 指定文件路径
file_path = 'data_post.json'
import json

# 将数据写入 JSON 文件
with open(file_path, 'w', encoding='utf-8') as json_file:
    json.dump(data_json, json_file, ensure_ascii=False, indent=4)


665b3a320000000005007243
5
5
[0, 2, 3, 4, 5]
[(0, '66d81fd2000000001202f555'), (1, '66b6e4a8000000000f00e4cf'), (2, '66af585a000000001c03cce5'), (3, '66893597000000000303997b')]
[0, 0, 0, 0]
m_list [0, 0, 0, 0, 0, 0, 0]
